# Classical Simulation

Many quantum information subroutines are reversible classical operations. The catch is that they are operated on superpositions of classical bitstrings. However, we can unit test these operations by checking against specific bitstrings through the `call_classically` bloqs protocol.

In [ ]:
from typing import *

import numpy as np
from attrs import frozen
from numpy.typing import NDArray

from cirq_qubitization.quantum_graph.bloq import Bloq
from cirq_qubitization.quantum_graph.composite_bloq import CompositeBloqBuilder
from cirq_qubitization.quantum_graph.fancy_registers import FancyRegister, FancyRegisters, Side
from cirq_qubitization.jupyter_tools import show_bloq

## CNOT

The simplest classical gate is the controlled not. This flips the target bit if `ctrl` is set. We can implement the `on_classical_vals` method to encode this behavior.

In [ ]:
@frozen
class CNOTExample(Bloq):
    @property
    def registers(self) -> 'FancyRegisters':
        return FancyRegisters.build(ctrl=1, target=1)

    def on_classical_vals(
        self, ctrl: NDArray[np.uint8], target: NDArray[np.uint8]
    ) -> Dict[str, NDArray[np.uint8]]:
        target_out = (ctrl + target) % 2
        return {'ctrl': ctrl, 'target': target_out}

In [ ]:
CNOTExample().call_classically(ctrl=1, target=0)

In [ ]:
import itertools
for c, t in itertools.product([0,1], repeat=2):
    ret = CNOTExample().apply_classical(ctrl=c, target=t)
    print(f'{c}{t} -> {ret["ctrl"]}{ret["target"]}')

In [ ]:
from cirq_qubitization.quantum_graph.graphviz import ClassicalSimGraphDrawer

in_data = {
    'ctrl': np.array([1], dtype=np.uint8),
    'target': np.array([0], dtype=np.uint8),
}

drawer = ClassicalSimGraphDrawer(CNOTExample(), in_data)
drawer.get_svg()

In [ ]:
bb = CompositeBloqBuilder()
q0 = bb.add_register('q0', 1)
q1 = bb.add_register('q1', 1)
q0, q1 = bb.add(CNOTExample(), ctrl=q0, target=q1)
q1, q0 = bb.add(CNOTExample(), ctrl=q1, target=q0)
q0, q1 = bb.add(CNOTExample(), ctrl=q0, target=q1)
cbloq = bb.finalize(q0=q0, q1=q1)

in_data = {
    'q0': np.array([1], dtype=np.uint8),
    'q1': np.array([0], dtype=np.uint8),
}

drawer = ClassicalSimGraphDrawer(cbloq, in_data)
drawer.get_svg()